## 32. Longest Valid Parentheses


Given a string containing just the characters `'('` and `')'`, find the length of the longest valid (well-formed) parentheses substring.

For `"(()"`, the longest valid parentheses substring is `"()"`, which has length = 2.

Another example is `")()())"`, where the longest valid parentheses substring is `"()()"`, which has length = 4.


DP问题：

定义 `dp(i)` 为 i 位置的最长括号结构的 长度（最优解）， 在最优解满足一下条件：

- `s(i) == '('` , `dp(i) = 0`
- `s(i) == ')'`
    - `s(i-1) == '('`,  `dp(i) = dp(i-2) + 2`
    - `s(i-1) == ')'`
        - `s(i-1-dp(i-1)) == '('`, `dp(i) = dp(i-1) + 2 + dp(i-2-dp(i-1))`
        - `s(i-1-dp(i-1)) == ')'`, `dp(i) = 0`
        



In [11]:
class Solution:
    def longestValidParentheses(self, s):
        """
        :type s: str
        :rtype: int
        """
        dp = [0] * len(s)
        maxlen = 0
        for i in range(1, len(s)):
            if s[i] == ')':
                if s[i-1] == '(':
                    dp[i] = dp[i-2] + 2
                elif i > dp[i-1] and s[i-1-dp[i-1]] == '(':
                    dp[i] = dp[i-1] + 2 + dp[i-2-dp[i-1]]
            print(i, dp[i], s[i], s[i-1-dp[i-1]])
            maxlen = max(maxlen, dp[i])
            
        return maxlen
    
# Solution().longestValidParentheses(')()())')
Solution().longestValidParentheses("(()))())(")

1 0 ( (
2 2 ) (
3 4 ) (
4 0 ) (
5 0 ( )
6 2 ) (
7 0 ) )
8 0 ( )


4

上面是典型的DP做法。还有更巧妙的办法。

其实这个括号结构有点特殊。仔细看会发现，它的模式就是：

```
expr = '()' |  '(' + expr + ')' | expr + expr
```

除去最后的 `expr + expr`，  `expr` 总是对称的。

一种想法是，从左向右scan一遍， 对 `(` 和 `)` 计数， 如果`)`的数量超过了 `(`的数量，说明不是有效结构，重置计数； 如果 `)` 的数量小于 `(` 的数量，则说明还有一些括号需要不全； 如果二者相等，则说明结构合理。

如果 `(` 的数量总是超过 `)`， 从右向左scan一遍，就可以解决问题。


In [13]:
class Solution:
    def longestValidParentheses(self, s):
        """
        :type s: str
        :rtype: int
        """
        maxlen = 0
        left = right = 0
        for c in s:
            if c == '(':
                left += 1
            else:
                right += 1
            if left == right:
                maxlen = max(maxlen, right * 2)
            elif right > left:
                left = right = 0
        left = right = 0
        for c in s[::-1]:
            if c == '(':
                left += 1
            else:
                right += 1
                
            if left == right:
                maxlen = max(maxlen, right * 2)
            elif left > right:
                left = right = 0
        return maxlen
# Solution().longestValidParentheses(')()())')
Solution().longestValidParentheses("(()))())(")

4

## 64. Minimum Path Sum

Given a m x n grid filled with non-negative numbers, find a path from top left to bottom right which minimizes the sum of all numbers along its path.

Note: You can only move either down or right at any point in time.

```
Example 1:
[[1,3,1],
 [1,5,1],
 [4,2,1]]
Given the above grid map, return 7. Because the path 1→3→1→1→1 minimizes the sum.

```

典型的DP问题。很简单， `dp[i, j] = min(dp[i-1, j], dp[i, j-1]) + gird[i, j]`

算法复杂度 $O(nm)$， 空间复杂度 $O(nm)$。 其实可以只用一行，简化空间复杂度。

In [19]:

class Solution:
    def minPathSum(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        dp = []
        s = 0
        for x in grid[0]:
            s += x
            dp.append(s)
        # print(dp)    
        for i in range(1, len(grid)):
            dp[0] += grid[i][0]
            for j in range(1, len(dp)):
                dp[j] = min(dp[j-1], dp[j]) + grid[i][j]
            # print(dp)
        return dp[-1]
    
Solution().minPathSum([[1,3,1], [1, 5, 1], [4, 2, 1]])

7

## 72. Edit Distance

Given two words word1 and word2, find the minimum number of steps required to convert word1 to word2. (each operation is counted as 1 step.)

You have the following 3 operations permitted on a word:

a) Insert a character
b) Delete a character
c) Replace a character

题目大意是 两个字符串， 只允许3种操作，问最少需要多少操作可以将word1 转成word2？

也是动态规划问题：令 `dp[i,j]` 是问题 `word1[0:i]` 和 `word2[0:j]` 的解。则：

1. `dp[i][0] = i` ，一个空字符串和一个字符串，单纯用插入操作就可以解决。
2. `dp[0][j] = j` ，同上。
3. 如果`word1[i - 1] = word2[j - 1]`， 则 `dp[i][j] = dp[i - 1][j - 1]`
4. 否则 `dp[i][j] = min(dp[i - 1][j - 1] + 1, dp[i - 1][j] + 1, dp[i][j - 1] + 1)`
    1. 拆开看就是 替换、插入或者删除 三种情况的最优选择。

上面的过程可以对空间进行优化：
```
(i-1, j-1),  (i-1, j)
(i,   j-1),  (i,   j)
```
可以看出就是一个网格结构。

所以跟上个一样，维护一行就行了。 并且还可以选择只维护短的word所代表的行。


```
prev,   cache
dp[j],  dp[j+1]
```

In [31]:
class Solution:
    def minDistance(self, word1, word2):
        """
        :type word1: str
        :type word2: str
        :rtype: int
        """
        n, m = len(word1), len(word2)
        if m > n:
            n, m, word1, word2 = m, n, word2, word1
        if m == 0:
            return n
        
        dp = list(range(m+1))
        for i in range(n):
            prev = i
            dp[0] = i + 1
            for j in range(m):
                cache = dp[j+1]
                if word1[i] == word2[j]:
                    dp[j+1] = prev
                else:
                    dp[j+1] = min(prev, dp[j], cache) + 1
                prev = cache
                
        return dp[m]
    
# Solution().minDistance('abc', 'efg')
Solution().minDistance('a', 'a')
        

0

## 87. Scramble String

s1 和 s2 是两个字符串

如果 s1 的树状表示，在叶节点翻转后组成的字符串表示为 s2，则 s2 是 s1的 scramble。。。

也是DP问题。 关键是这个树状结构定义怎么定义。

思路非常简单， 一个 `s1, s2` 问题，可以划分成 `4*n` 个子问题： `s1[:i], s2[:i]`和 `s1[i:], s2[i:]`， 或者 `s1[:i], s2[-i:]` 和 `s1[-i:], s2[:i]`。 但是这样递归下去，感觉规模非常庞大。

考虑所有索引位置， 大约有 $n^4$ 个子问题。 但是大部分都无效，因为长度不一致。

In [56]:

class Solution:
    # @return a boolean
    def isScramble(self, s1, s2):
        n, m = len(s1), len(s2)
        if n != m or sorted(s1) != sorted(s2):
            return False
        if n < 4 or s1 == s2:
            return True
        f = self.isScramble
        for i in range(1, n):
            if f(s1[:i], s2[:i]) and f(s1[i:], s2[i:]) or \
               f(s1[:i], s2[-i:]) and f(s1[i:], s2[:-i]):
                return True
        return False
    
# Solution().isScramble("great", "rgeat")
Solution().isScramble("abcdefghijklmn", "efghijklmncadb")

False

In [58]:
class Solution:
    # @return a boolean
    def isScramble(self, s1, s2):
        n, m = len(s1), len(s2)
        if n != m:
            return False
        
        def dp(ss1, ss2):
            # 这是可以节省运行时间的关键：如果字符都不匹配，一定是错误的
            if sorted(ss1) != sorted(ss2):
                return False
            sn = len(ss1)
            if sn < 4 or ss1 == ss2:
                return True
            for i in range(1, sn):
                if (dp(ss1[:i], ss2[:i]) and dp(ss1[i:], ss2[i:])) or \
                       (dp(ss1[:i], ss2[-i:]) and dp(ss1[i:], ss2[:-i])):
                    return True
            return False        
        return dp(s1, s2)
    
# Solution().isScramble("abc", "bca")
Solution().isScramble("abcdefghijklmn", "efghijklmncadb")

False

## 95. Unique Binary Search Trees II

构建所有的二分查找树。对于 `1...n`

这也是个DP问题。

$$ DP(1, n) = \sum_i{DP(1, i-1) + DP(i+1, n)} $$

实际上也是一个DFS 搜索问题。

In [90]:
class Solution(object):
    def generateTrees(self, n):
        """
        :type n: int
        :rtype: List[TreeNode]
        """
        if n == 0:
            return []
        return self.dfs(1, n+1)
        
    def dfs(self, start, end):
        if start == end:
            return None
        result = []
        for i in range(start, end):
            for l in self.dfs(start, i) or [None]:
                for r in self.dfs(i+1, end) or [None]:
                    node = TreeNode(i)
                    node.left, node.right  = l, r
                    result.append(node)
        return result
    
    
def print_solution(n):
    def print_nodes(node):
        if node is None:
            print("null", end=',')
        else:
            print(node.val, end=',')
            print_nodes(node.left)
            print_nodes(node.right)
        
    list_trees = Solution().generateTrees(n)
    print(list_trees)
    for tree in list_trees:
        print_nodes(tree)
        print()

print_solution(3)

[<__main__.TreeNode object at 0x110ab86d8>, <__main__.TreeNode object at 0x110ab8cc0>, <__main__.TreeNode object at 0x110ab86a0>, <__main__.TreeNode object at 0x110ab84e0>, <__main__.TreeNode object at 0x110ab8390>]
1,null,2,null,3,null,null,
1,null,3,2,null,null,null,
2,1,null,null,3,null,null,
3,1,null,2,null,null,null,
3,2,1,null,null,null,null,


下面是 generator版本的

In [93]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None
    
    def __str__(self):
        return str(self.val)

class Solution(object):
    def generateTrees(self, n):
        """
        :type n: int
        :rtype: List[TreeNode]
        """
        if n == 0:
            return []
        
        def trees(i, j):
            if i > j:
                yield None
            for k in range(i, j+1):
                for left in trees(i, k-1):
                    for right in trees(k+1, j):
                        node = TreeNode(k)
                        node.left, node.right = left, right
                        yield node              
            
        return list(trees(1, n))
    
# Solution().generateTrees(3)
print_solution(3)

[<__main__.TreeNode object at 0x110b58780>, <__main__.TreeNode object at 0x110b58898>, <__main__.TreeNode object at 0x110b58940>, <__main__.TreeNode object at 0x110b589e8>, <__main__.TreeNode object at 0x110b58a90>]
1,null,2,null,3,null,null,
1,null,3,2,null,null,null,
2,1,null,null,3,null,null,
3,1,null,2,null,null,null,
3,2,1,null,null,null,null,


## 97. Interleaving String

交织string：给定 s1, s2, s3， 判断 s3 是否由 s1 和 s2 交织而成

例如：
```
Given:
s1 = "aabcc",
s2 = "dbbca",

When s3 = "aadbbcbcac", return true.
When s3 = "aadbbbaccc", return false.
```

一开始有一个简单的想法： 就是从 s3中划掉 s1的字符，剩下的如果和 s2相同，就是交织而成的。其实不对，有个反例：
```
s1="abc"
s2="bcd"
s3="abcbdc"
```

还是老老实实用 DP。

DP的思路其实也很简单：令 `DP(i, j)` 表示 `s1[i:]` 、`s2[j:]` 和 `s3[i+j:]`的解

- 如果 `s1[i] == s2[j] == s3[i+j]` ， 则 `DP(i, j) = DP(i+1, j) or DP(i, j+1)`
- 如果 `s1[i] != s2[j]`， 
    - 如果 `s3[i+j] == s1[i]`， 则 `DP(i, j) = DP(i+1, j)`
    - 如果 `s3[i+j] == s2[j]`， 则 `DP(i, j) = DP(i, j+1)`
    - 都不相同， 则 `DP(i, j) = False`
    
反向的思路：


DP的思路其实也很简单：令 `DP(i, j)` 表示 `s1[:i]` 、`s2[:j]` 和 `s3[:i+j]`的解

- 如果 `s1[i] == s2[j] == s3[i+j]` ， 则 `DP(i, j) = DP(i-1, j) or DP(i, j-1)`
- 如果 `s1[i] != s2[j]`， 
    - 如果 `s3[i+j] == s1[i]`， 则 `DP(i, j) = DP(i-1, j)`
    - 如果 `s3[i+j] == s2[j]`， 则 `DP(i, j) = DP(i, j-1)`
    - 都不相同， 则 `DP(i, j) = False`

- `DP(i, 0) = s1[:i] == s3[:i]`
- `DP(0, j) = s2[:j] == s3[:j]`

    
注意这也是一个网格样式的DP（正向和反向都一样）

```
x         (i-1, j)
(i, j-1)  (i, j)
```


这里注意一个大trick：  就是索引的问题： 注意： `a` 和 `b` 可以拼成 `ab`，那么 索引呢？ ， 0, 0，0，永远到不了1。 使用这种索引方式有很大缺陷的： `i in range(n), j in range(m)`，则永远无法判断 s3的最后一个字符。必须用正常索引方式： `i in range(1, n+1), j in range(1, m+1), k = i + j`，实际用在字符串索引时，都减1

In [125]:
class Solution:
    def isInterleave(self, s1, s2, s3):
        """
        :type s1: str
        :type s2: str
        :type s3: str
        :rtype: bool
        """
        n, m, l = len(s1), len(s2), len(s3)
        if n + m != l:
            return False
        if m > n:
            n, m, s1, s2 = m, n, s2, s1
            
        if m == 0:
            return s1 == s3
        
        dp = [False] * (m + 1)
        for i in range(n+1):
            for j in range(m+1):
                if (i == 0 and j == 0):
                    dp[j] = True
                elif (i == 0):
                    dp[j] = dp[j-1] and s2[j-1] == s3[j-1]
                elif (j == 0):
                    dp[j] = dp[j] and s1[i-1] == s3[i-1]
                else:
                    dp[j] = (dp[j] and s1[i-1] == s3[i+j-1]) or (dp[j-1] and s2[j-1] == s3[i+j-1])
        return dp[-1]
    
print(Solution().isInterleave('abc', 'bcd', 'abcbdc'))
print(Solution().isInterleave('aabcc', 'dbbca', 'aadbbcbcac'))
print(Solution().isInterleave('aabcc', 'dbbca', 'aadbbbaccc'))
print(Solution().isInterleave('', '', ''))
print(Solution().isInterleave('a', '', 'a'))
print(Solution().isInterleave('a', 'b', 'ab'))
                

True
True
False
True
True
True


## 115. Distinct Subsequences

Given a string S and a string T, count the number of distinct subsequences of S which equals T.

A subsequence of a string is a new string which is formed from the original string by deleting some (can be none) of the characters without disturbing the relative positions of the remaining characters. (ie, "ACE" is a subsequence of "ABCDE" while "AEC" is not).

Here is an example:
```
S = "rabbbit", T = "rabbit"

Return 3.
```

题目大意是 ：  问 S有多少个不同的 子串等于 T。 这里不同意思是截取的位置不同。

这里是解法：<https://leetcode.com/problems/distinct-subsequences/discuss/37327/Easy-to-understand-DP-in-Java>

不看真不知道，解释的很清楚。是个DP问题。

定义 `DP(i+1, j+1)` 是 问题 `S[:i]` 和 `T[:j]`的解。

- `T == ''`， `DP(i, 0) = 1`，因为空串是任意字符串的子串，且可以认为是唯一。
- `T != '', S = ''`, `DP(0, j) = 0`
- 如果 `S[i] == T[j]`, `DP(i+1, j+1) = DP(i,j) + DP(i, j+1)`， 如果字符相同， `S[:i-1]` 中有多少`T[:j-1]`子串，分别附上这个相同字符； 另外还有， `S[:i-1]`中有多少 `T[:j]`子串，

这样看就很清楚：

```
  S 0123....i
T +----------+
0 |1111111111|
1 |0         |
2 |0         |
. |0         |
. |0         |
j |0         |
```
比如 `s = 'acdbabefbc', t = 'abc'`，则：

```
   s  =  acdbabefbc
'' dp = 11111111111     # 因为 ""空字符串是所有字符串的子串，只取1
'a'   = 01111222222     # 'a' 出现的时候，开始计数
'b'   = 00001           # 'b' 出现第一次，必须前面有 'a'才算第一次。
      = 0000113         # 'b' 出现第二次，感觉像个斐波那契问题：这个'b'和前面两个'a' 构成2个子串，在加上前一个'ab'，就是3个
      = 00001133355     # 'b' 出现第三次，这个'b' 和前面两个'a' 构成2个子串，再加上前面3个， 就是5个。
'c'   = 00_             # 'c' 出现第一次，因为前面没有'ab'，所以不计数
      = 00000000005     # 'c' 出现第二次，因为前面有5个'ab'子串，所以，总共有5个。
```

```
   s  =  rabbbbit
'' dp = 111111111     # 因为 ""空字符串是所有字符串的子串，只取1
'r'   = 011111111     # 'r' 出现的时候，开始计数
'a'   = 001111111     # 
'b'   = 000123444     # 
'b'   = 000013666     # 注意这里，计算的时候用来左上角，而不是正上方。
'i'   = 000000066
```


In [131]:
class Solution:
    def numDistinct(self, s, t):
        """
        :type s: str
        :type t: str
        :rtype: int
        """
        n, m = len(s), len(t)
        if n < m:
            return 0
        if n == m:
            return int(s == t)
        
        dp = [1] * (n+1)
        print(dp)
        for j in range(m):
            dp[0] = 0
            for i in range(n):
                if s[i] == t[j]:
                    dp[i+1] += dp[i]
                else:
                    dp[i+1] = dp[i]
            print(dp)
        return dp[-1]

# Solution().numDistinct('acdbabefbc', 'abc')
Solution().numDistinct("rabbbit", "rabbit")

[1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 1, 2, 3, 3, 3]
[0, 0, 0, 1, 3, 6, 6, 6]
[0, 0, 0, 0, 0, 0, 6, 6]
[0, 0, 0, 0, 0, 0, 0, 6]


6